# Order a Basemap Using the Orders API & SDK via Quad ID
---

## Overview ##
---
Download Basemap quads based on the mosaic name and quad IDs.

For example:
point_reyes_rolling_normalized_2020-04-30_mosaic

and

325-1260,326-1260,325-1259,326-1259


<a name="Get-quad-IDs"></a> 
## Get quad IDs
---

You can get a basemap ID from Planet Explorer or the Basemap Viewer, as described in jupyter-notebooks/quad_tutorial/basemaps_ordersv2_aoi.ipynb. Each basemap is broken down into quads. To get the quad IDs for a basemap:

1. In the Basemap Viewer, enter your basemap ID, such as point_reyes_rolling_normalized_2020-04-30_mosaic
2. On the resulting basemap, select the arrow to view the quads and scenes, for example: 325-1260,326-1260,325-1259,326-1259


<a name="how-to-get-started"></a> 
## Start coding

---
Next set up a session by importing needed Python packages, pulling in your API Key, and make an initial request (to retrieve the Orders API parameters) to confirm a connection with the server.

In [ ]:
import planet
import os
import requests
from requests.auth import HTTPBasicAuth
import json


In [ ]:
ORDERS_API_URL = 'https://api.planet.com/compute/ops/orders/v2'
API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
orders_url = 'https://api.planet.com/compute/ops/orders/v2'
auth = HTTPBasicAuth(API_KEY, '')
(fix this)

## Use Basemaps API to get the Basemap and quad IDs
---

Use the Basemap APIs to retrieve the basemap you want and the quad IDs. You can get the base URL you'll need to communicate with the Basemap API service:

1. Go to the [Basemap API reference](https://developers.planet.com/docs/basemaps/reference/) at https://developers.planet.com/docs/basemaps/reference/. 
2. Under List Mosaics, select GET /mosaics.

A dropdown UI element appears with the URL to use:

https://api.planet.com/basemaps/v1/mosaics


#### Search for the mosaic ID to get its quad IDs

Here, we are using the full name of the mosaic, but with the `name__contains` field, you can use a partial name based on the names of the mosaics available through your account.

Declare the search string for `requests.get()`. You will use the dictionary you create with the mosaic name to pass to the requests.get() function.


In [ ]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'
headers = {'content-type': 'application/json'}

basemap_params = {
    'name__contains': 'point_reyes_rolling_normalized_2020-04-30_mosaic'
}

basemap_service_response = requests.get(
    url=BASEMAP_API_URL, params=basemap_params, auth=auth, headers=headers)

basemaps = basemap_service_response.raise_for_status()
if basemap_service_response.status_code != 204:
    basemaps = json.loads(basemap_service_response.text)
print(basemaps)


In [ ]:
mosaic_ID = basemaps['mosaics'][0]['id']
mosaic_box = basemaps['mosaics'][0]['bbox']
mosaic_box_str = ','.join(map(str, mosaic_box))

quad_params = {
    'mosaic_id': mosaic_ID,
    'bbox': mosaic_box_str,
}

quads_url = "{}/{}/quads".format(BASEMAP_API_URL, mosaic_ID)
quadServiceResponse = requests.get(
    url=quads_url, params=quad_params, auth=auth, headers=headers)
quads = quadServiceResponse.json()
items = quads['items']

quadIds = []
for i in items:
    quadId = i['id']
    quadIds.append(quadId)


### Create an order packet
---
Package up the details of your order in a JSON object and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. 

In [ ]:
order_params = {
    "name": "basemap order with quad_ids",
    "source_type": "basemaps",
    "products": [
        {
            "mosaic_name": "point_reyes_rolling_normalized_2020-04-30_mosaic",
            "quad_ids": quadIds
        }
    ],
    "tools": [
        {
            "reproject": {
                "projection": "EPSG:4326",
                "resolution": 3.0 / 111000.0,
                "kernel": "cubic"
            }
        },
        {
            "bandmath": {
                "b1": "b1",
                "b2": "b2",
                "b3": "b3",
                "b4": "arctan(b1)",
                "b5": "(b4-b3)/(b4+b3)",
                "pixel_type": "32R"
            }
        }
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "notebook-practice",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/",
        }
    }
}


### Create a session with SDK and poll for success

Here, we are creating a session with the SDK and creating the order using the order parameters. Then, we are polling for success. This may take a few minutes to complete.



In [ ]:
with planet.reporting.StateBar(state='creating') as reporter:
    async with planet.Session() as sess:
        client = planet.OrdersClient(sess)
        # perform operations here
        order = await client.create_order(order_params)
        reporter.update(state='created', order_id=order['id'])
        await client.wait(order['id'], callback=reporter.update_state,
                          max_attempts=0)
